In [1]:
%reload_ext autoreload
%autoreload 2

from qwanta import Tuner, QuantumNetwork, Configuration, Xperiment
from qwanta import QuantumNetwork, Configuration, Xperiment, Tuner
import pandas as pd
import ast
import numpy as np
import networkx as nx
from qwanta.Qubit import PhysicalQubit
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def adjusted_rate(rl, dl, ds, loss=0.1):
    return rl*10**((dl - ds)*loss/10)

minimal_edges_infos = [(0.0001, 100, 100), (0.0001, 175, 25), 
                       (0.0001, 150, 50) ,(0.0001, 125, 75)]
adjusted_rate_list = [adjusted_rate(*i) for i in minimal_edges_infos]

adjusted_rate_list

[0.0001, 0.0031622776601683794, 0.001, 0.000316227766016838]

In [3]:
# Hyperparameters

loss = 0.1 # dB/km
depo_prob = 0
gate_error = 0.1 
measurement_error = 0
memory_time = 1 # second

num_hops = 2
num_nodes = num_hops + 1

# 3 Equidistant nodes
# node_info = {f'Node {i}': {'coordinate': (int(i*100), 0, 0)} for i in range(num_nodes)}

# 3 non-equidistant nodes
node_info = {'Node 0': {'coordinate': (0, 0, 0)},
 'Node 1': {'coordinate': (50, 0, 0)},
 'Node 2': {'coordinate': (200, 0, 0)}}

# # default edge info
# edge_info = {
#     (f'Node {i}', f'Node {i+1}'): {
#     'connection-type': 'Space',
#     'depolarlizing error': [1 - depo_prob, depo_prob/3, depo_prob/3, depo_prob/3],
#     'loss': loss,
#     'light speed': 300000,
#     'Pulse rate': 0.0001,
#     f'Node {i}':{
#         'gate error': gate_error,
#         'measurement error': measurement_error,
#         'memory function': memory_time
#     },
#     f'Node {i+1}':{
#         'gate error': gate_error,
#         'measurement error': measurement_error,
#         'memory function': memory_time
#     },
#     }
# for i in range(num_hops)}

# edge with adjust pulse rate
edge_info = {('Node 0', 'Node 1'): 
        {'connection-type': 'Space',
        'depolarlizing error': [1, 0.0, 0.0, 0.0],
        'loss': 0.1,
        'light speed': 300000,
        'Pulse rate': adjusted_rate(0.0001, 150, 50, loss=loss),
        'Node 0': {'gate error': 0.1, 'measurement error': 0, 'memory function': 1},
        'Node 1': {'gate error': 0.1, 'measurement error': 0, 'memory function': 1}},

        ('Node 1', 'Node 2'): {'connection-type': 'Space',
        'depolarlizing error': [1, 0.0, 0.0, 0.0],
        'loss': 0.1,
        'light speed': 300000,
        'Pulse rate': 0.0001,
        'Node 1': {'gate error': 0.1, 'measurement error': 0, 'memory function': 1},
        'Node 2': {'gate error': 0.1, 'measurement error': 0, 'memory function': 1}}}

exps = Xperiment(
    timelines_path = 'exper_id3_selectedStats_2hops.xlsx',
    nodes_info_exp = node_info,
    edges_info_exp = edge_info,
    gate_error = gate_error,
    measurement_error = measurement_error,
    memory_time = memory_time,
    strategies_list=['0G']
)

results = exps.execute()

In [15]:
def visualize(result):
    to_print = ['fidelity', 'simulation_time', 'Resources Produced', 'Base Resources Attempt', 'Resource Used in Fidelity Estimation', 'Time used', 'Fidelity Estimation Time']
    # to_print = ['fidelity', 'Base Resources Attempt', 'Fidelity Estimation Time']
    for key in to_print:
        print(f'{key}: {result[key]}')


In [16]:
visualize(results['0G'])

fidelity: 0.7951666666666666
simulation_time: None
Resources Produced: {'Node 0-Node 1': {'Physical': 9001}, 'Node 1-Node 2': {'Physical': 9003}, 'Node 0-Node 2': {'k': 9001}}
Base Resources Attempt: 309690
Resource Used in Fidelity Estimation: 9000
Time used: 28.675933333319115
Fidelity Estimation Time: 28.674933333319114


In [25]:
throughput = 9000/(results['0G']['Time used'] + results['0G']['Fidelity Estimation Time'])
throughput

156.92875318369724

In [6]:
data = {
    'Experiment name': [
        'Equidistant nodes', 
        'Non-equidistant nodes 50, 200', 
        'Non-equidistant nodes 50, 200, adjusted pulse rate'
        
        ],
    'Fidelity': [
        0.813, 
        0.7670000000000001, 
        0.7815000000000001
        ],
    'Base Resources Attempt': [
        179439, 307780, 315801],
    'Fidelity Estimation Time': [
        9.232366666658779, 
        27.977499999948133,
        29.50823333332347
        ]
}

df = pd.DataFrame(data)
df

,Experiment name,Fidelity,Base Resources Attempt,Fidelity Estimation Time
0,Equidistant nodes,0.8130,179439,9.232367
1,"Non-equidistant nodes 50, 200",0.7670,307780,27.977500
2,"Non-equidistant nodes 50, 200, adjusted pulse ...",0.7815,315801,29.508233
